### Task [101](https://github.com/salgo60/ProjectOutdoorGyms/issues/101) "uppdater Örebro utegym"
Verkar som det händer lite och finns API data se även [Notebook Stockholm](https://github.com/salgo60/ProjectOutdoorGyms/issues/99)

* API [öppna kartdata Örebro](http://data.karta.orebro.se/)
  * [API call](http://data.karta.orebro.se/opendataapi/v1/search/utegym)
  * [API variant med layer=500](http://data.karta.orebro.se/opendataapi/v1/layers/500)
  * API variant med WGS84 [v1/layers/500?srs=EPSG:4326-> ](http://data.karta.orebro.se/opendataapi/v1/layers/500?srs=EPSG:4326)
  
* [denna notebook](https://github.com/salgo60/ProjectOutdoorGyms/blob/main/Jupyter/Örebro%20outdoor%20gym.ipynb)


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-05-27 08:33:43.436384


In [2]:
url1="http://data.karta.orebro.se/opendataapi/v1/search/utegym"
url2="http://data.karta.orebro.se/opendataapi/v1/layers/500"  
url3WGS84="http://data.karta.orebro.se/opendataapi/v1/layers/500?srs=EPSG:4326"

url=url3WGS84
import urllib3, json
import pandas as pd
from tqdm import tqdm
http = urllib3.PoolManager()
try:
#    r = http.request('POST', url,
#                 headers={'Content-Type': 'application/json'}
#                    )
    r = http.request('GET', url)
except:
    print ("Error and r.status\t",r.status, url)
print("Status\t",r.status,url)
#print(r.data)

Status	 200 http://data.karta.orebro.se/opendataapi/v1/layers/500?srs=EPSG:4326


In [3]:
print(type(r.data)) # Bytes --> need decode 
data = r.data.decode('utf-8')
print(type(data))
data = json.loads(r.data.decode('utf-8'))
#for d in data["features"]:
#    print(d)

<class 'bytes'>
<class 'str'>


In [4]:
Utegymdata = pd.DataFrame(data["features"]) 
Utegymdata.keys() 
Utegymdata= pd.json_normalize(data["features"])
Utegymdata.keys()  
Utegymdata


,type,geometry.type,geometry.coordinates,properties.ID,properties.TITEL,properties.BESKRIVNING,properties.LINK,properties.PUBLICERAD
0,Feature,Point,"[15.257027349323, 59.2771491218136]",1423,Alnängarnas utegym,"Vid Svartån, mellan Alnängsbadet och Osetbron ...",http://www.orebro.se/13503.html,Ja
1,Feature,Point,"[15.1992074345839, 59.2897499398519]",1601,Utegym Baronbackarma,Utegym,www.orebro.se/utegym,Ja
2,Feature,Point,"[15.2271728435398, 59.2867914577759]",1602,Utegym Bergkällebacken,Utegym,www.orebro.se/utegym,Ja
3,Feature,Point,"[15.244370603546, 59.2340711282985]",1603,Utegym Saxons park,Utegym,www.orebro.se/utegym,Ja
4,Feature,Point,"[15.2319917959567, 59.2737936201181]",1604,Utegym Storaholmen,Utegym,www.orebro.se/utegym,Ja
5,Feature,Point,"[15.1944072095474, 59.2698288432734]",1605,Utegym Ånäsparken,Utegym,www.orebro.se/utegym,Ja
6,Feature,Point,"[15.2153126214241, 59.255202513509]",1621,Utegym Grönpepparparken,Utegym,www.orebro.se/utegym,Ja
7,Feature,Point,"[15.2394460296104, 59.2876737883058]",2202,Utegym Venaspåret,Utegym,https://www.orebro.se/utegym,Ja
8,Feature,Point,"[15.1451998719678, 59.2765126961365]",2221,Utegym Karlslund,Utegym,https://www.orebro.se/utegym,Ja
9,Feature,Point,"[15.2519367427569, 59.2448154147077]",2241,Utegym Markaspåret,Utegym,https://www.orebro.se/utegym,Ja


### Status JSON 
* see [feedback]
* fular och lägger in properties.ID precis som jag gjorde med Stockholm
* compare [notebook Stockholm](https://github.com/salgo60/ProjectOutdoorGyms/blob/main/Jupyter/Stockholm%20outdoor%20gym.ipynb) with 100 outdoor gym


In [5]:
Utegymdata.to_csv("Örebroutegym.csv")

### SPARQL Wikidata
Hämta utegym i WD för Örebro kommun [wd:Q297718](https://www.wikidata.org/wiki/Q297718?uselang=sv)
* [SPARQL](https://w.wiki/5D6N) Wikidata 

In [6]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

queryOrebro = """#title:Outdoor gyms Örebro in WD
SELECT distinct ?inception ?item ?itemLabel ?www ?id ?API  WHERE {
  ?item wdt:P6104 wd:Q107186275.
  OPTIONAL {?item wdt:P571 ?inception}
  BIND(URI(CONCAT("https://apigw.stockholm.se/noauth/virtualhittaservicedmz/rest/serviceunits/",?servicenr)) AS ?API)
                 
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en,sv,de" . 
  }
  
OPTIONAL{?item wdt:P571 ?date}
OPTIONAL { ?item wdt:P18 ?img}
?item wdt:P131 wd:Q297718  # Örebro kommun
OPTIONAL { ?item wdt:P856 ?www. }
OPTIONAL { ?item wdt:P625 ?coord. }
OPTIONAL { ?item wdt:P217 ?id. }
} order by desc(?date)"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDtotOutdoorGym = get_sparql_dataframe(endpoint_url, queryOrebro)

WDtotOutdoorGym


,inception,item,itemLabel,www,id,API
0,2019-10-10T00:00:00Z,http://www.wikidata.org/entity/Q107810233,Tengvallsgatan utegym,https://www.obo.se/nyheter-press/invigning-av-...,None,None
1,None,http://www.wikidata.org/entity/Q107250681,"Alnängarna, outdoorgym",https://www.orebro.se/kultur--fritid/motion--i...,1423,None
2,None,http://www.wikidata.org/entity/Q107250740,"Baronbackarna, outdoorgym",https://www.orebro.se/utegym,1601,None
3,None,http://www.wikidata.org/entity/Q107250866,"Ånäsparken, outdoorgym",https://www.orebro.se/kultur--fritid/motion--i...,1605,None
4,None,http://www.wikidata.org/entity/Q107407390,"Bergkällebacken, outdoor gym",https://www.orebro.se/utegym,1602,None
5,None,http://www.wikidata.org/entity/Q107407400,"Grönpepparparken, outdoor gym",https://www.orebro.se/utegym,1621,None
6,None,http://www.wikidata.org/entity/Q107407402,"Karlslund, outdoor gym",https://www.orebro.se/utegym,2221,None
7,None,http://www.wikidata.org/entity/Q107407403,"Lillåspåret, outdoor gym",https://www.orebro.se/utegym,2261,None
8,None,http://www.wikidata.org/entity/Q107407405,"Markaspåret, outdoor gym",https://www.orebro.se/utegym,2241,None
9,None,http://www.wikidata.org/entity/Q107407406,"Saxons park, outdoor gym",https://www.orebro.se/utegym,1603,None


In [7]:
WDtotOutdoorGym.isnull().sum()

inception    14
item          0
itemLabel     0
www           1
id            4
API          15
dtype: int64

In [8]:
WDOrebroNew = WDtotOutdoorGym.rename(columns={'id': 'properties.ID'})

In [9]:
WDOrebroNew

,inception,item,itemLabel,www,properties.ID,API
0,2019-10-10T00:00:00Z,http://www.wikidata.org/entity/Q107810233,Tengvallsgatan utegym,https://www.obo.se/nyheter-press/invigning-av-...,None,None
1,None,http://www.wikidata.org/entity/Q107250681,"Alnängarna, outdoorgym",https://www.orebro.se/kultur--fritid/motion--i...,1423,None
2,None,http://www.wikidata.org/entity/Q107250740,"Baronbackarna, outdoorgym",https://www.orebro.se/utegym,1601,None
3,None,http://www.wikidata.org/entity/Q107250866,"Ånäsparken, outdoorgym",https://www.orebro.se/kultur--fritid/motion--i...,1605,None
4,None,http://www.wikidata.org/entity/Q107407390,"Bergkällebacken, outdoor gym",https://www.orebro.se/utegym,1602,None
5,None,http://www.wikidata.org/entity/Q107407400,"Grönpepparparken, outdoor gym",https://www.orebro.se/utegym,1621,None
6,None,http://www.wikidata.org/entity/Q107407402,"Karlslund, outdoor gym",https://www.orebro.se/utegym,2221,None
7,None,http://www.wikidata.org/entity/Q107407403,"Lillåspåret, outdoor gym",https://www.orebro.se/utegym,2261,None
8,None,http://www.wikidata.org/entity/Q107407405,"Markaspåret, outdoor gym",https://www.orebro.se/utegym,2241,None
9,None,http://www.wikidata.org/entity/Q107407406,"Saxons park, outdoor gym",https://www.orebro.se/utegym,1603,None


In [10]:
Utegymdata

,type,geometry.type,geometry.coordinates,properties.ID,properties.TITEL,properties.BESKRIVNING,properties.LINK,properties.PUBLICERAD
0,Feature,Point,"[15.257027349323, 59.2771491218136]",1423,Alnängarnas utegym,"Vid Svartån, mellan Alnängsbadet och Osetbron ...",http://www.orebro.se/13503.html,Ja
1,Feature,Point,"[15.1992074345839, 59.2897499398519]",1601,Utegym Baronbackarma,Utegym,www.orebro.se/utegym,Ja
2,Feature,Point,"[15.2271728435398, 59.2867914577759]",1602,Utegym Bergkällebacken,Utegym,www.orebro.se/utegym,Ja
3,Feature,Point,"[15.244370603546, 59.2340711282985]",1603,Utegym Saxons park,Utegym,www.orebro.se/utegym,Ja
4,Feature,Point,"[15.2319917959567, 59.2737936201181]",1604,Utegym Storaholmen,Utegym,www.orebro.se/utegym,Ja
5,Feature,Point,"[15.1944072095474, 59.2698288432734]",1605,Utegym Ånäsparken,Utegym,www.orebro.se/utegym,Ja
6,Feature,Point,"[15.2153126214241, 59.255202513509]",1621,Utegym Grönpepparparken,Utegym,www.orebro.se/utegym,Ja
7,Feature,Point,"[15.2394460296104, 59.2876737883058]",2202,Utegym Venaspåret,Utegym,https://www.orebro.se/utegym,Ja
8,Feature,Point,"[15.1451998719678, 59.2765126961365]",2221,Utegym Karlslund,Utegym,https://www.orebro.se/utegym,Ja
9,Feature,Point,"[15.2519367427569, 59.2448154147077]",2241,Utegym Markaspåret,Utegym,https://www.orebro.se/utegym,Ja


In [11]:
OutdoorGym_diff = pd.merge(Utegymdata, WDOrebroNew, how='outer', indicator='Exist')

In [12]:
OutdoorGym_diff["Exist"].value_counts()

both          11
right_only     4
left_only      1
Name: Exist, dtype: int64

In [13]:
MissingGymsWD = OutdoorGym_diff[OutdoorGym_diff["Exist"] == "left_only"]

In [14]:
MissingGymsWD

,type,geometry.type,geometry.coordinates,properties.ID,properties.TITEL,properties.BESKRIVNING,properties.LINK,properties.PUBLICERAD,inception,item,itemLabel,www,API,Exist
11,Feature,Point,"[15.1581257739779, 59.2456473262426]",2301,Sanatorie Utegym,Utegym,www.orebro.se/utegym,Ja,NaN,NaN,NaN,NaN,NaN,left_only


In [22]:
MissingGymsKommun = OutdoorGym_diff[OutdoorGym_diff["Exist"] == "right_only"]
MissingGymsKommun

,type,geometry.type,geometry.coordinates,properties.ID,properties.TITEL,properties.BESKRIVNING,properties.LINK,properties.PUBLICERAD,inception,item,itemLabel,www,API,Exist
12,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,2019-10-10T00:00:00Z,http://www.wikidata.org/entity/Q107810233,Tengvallsgatan utegym,https://www.obo.se/nyheter-press/invigning-av-...,None,right_only
13,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,http://www.wikidata.org/entity/Q107452304,"Sörbybacken, outdoor gym",None,None,right_only
14,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,http://www.wikidata.org/entity/Q107452763,"Varbergaskogen, activity area",https://www.orebro.se/utegym,None,right_only
15,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,http://www.wikidata.org/entity/Q107718144,Brevens bruk outdoor gym,https://www.facebook.com/BrevensBrukAB/photos/...,None,right_only


In [23]:
MissingGymsKommun[["item","itemLabel","www"]]

,item,itemLabel,www
12,http://www.wikidata.org/entity/Q107810233,Tengvallsgatan utegym,https://www.obo.se/nyheter-press/invigning-av-...
13,http://www.wikidata.org/entity/Q107452304,"Sörbybacken, outdoor gym",None
14,http://www.wikidata.org/entity/Q107452763,"Varbergaskogen, activity area",https://www.orebro.se/utegym
15,http://www.wikidata.org/entity/Q107718144,Brevens bruk outdoor gym,https://www.facebook.com/BrevensBrukAB/photos/...


In [15]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-05-27 08:33:46.626075
Time elapsed (hh:mm:ss.ms) 0:00:03.191197
